In [1]:
#datasets
import pandas as pd
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import keras_tuner as kt  # Para la optimización de hiperparámetros

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

"""1.DATASET"""
# cargar dataset con los datos etiquetados
annotations = pd.read_csv('/tf/train/_annotations.csv')
# Codificador de clases
le = LabelEncoder()
annotations['class'] = le.fit_transform(annotations['class'])

"""2. EXTRACCION DE CARACTERISTICAS Y ETIQUETAS"""
def load_image_and_boxes(annotations, image_dir):
    images = []
    boxes = []
    labels = []
    
    for index, row in annotations.iterrows():
        image_path = os.path.join(image_dir, row['filename'])
        image = cv2.imread(image_path)  # Cargar en color
        height, width, _ = image.shape
        
        # Normalizar bounding boxes
        xmin = row['xmin'] / width
        ymin = row['ymin'] / height
        xmax = row['xmax'] / width
        ymax = row['ymax'] / height
        
        images.append(image)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(row['class'])
    
    return np.array(images), np.array(boxes), np.array(labels)

image_dir = '/tf/train/'
images, boxes, labels = load_image_and_boxes(annotations, image_dir)

"""3. CONSTRUCCION DEL MODELO CNN"""
def build_model(hp):
    def build_model(hp):
    input_shape = (1080, 1080, 1)  # Tamaño de la imagen de entrada
    num_classes = len(le.classes_)  # Número de clases en la salida
    
    input_image = layers.Input(shape=input_shape)
    
    # Normalización de imágenes de [0, 255] a [0, 1]
    x = layers.Rescaling(1./255)(input_image)
    
    # Bloque 1
    x = layers.Conv2D(
        filters=hp.Choice('filters_block1', [32, 64]), 
        kernel_size=hp.Choice('kernel_size_block1', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(
        filters=hp.Choice('filters_block1_2', [32, 64]), 
        kernel_size=hp.Choice('kernel_size_block1_2', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Bloque 2
    x = layers.Conv2D(
        filters=hp.Choice('filters_block2', [64, 128]), 
        kernel_size=hp.Choice('kernel_size_block2', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(
        filters=hp.Choice('filters_block2_2', [64, 128]), 
        kernel_size=hp.Choice('kernel_size_block2_2', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Bloque 3
    x = layers.Conv2D(
        filters=hp.Choice('filters_block3', [128, 256]), 
        kernel_size=hp.Choice('kernel_size_block3', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(
        filters=hp.Choice('filters_block3_2', [128, 256]), 
        kernel_size=hp.Choice('kernel_size_block3_2', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Bloque 4
    x = layers.Conv2D(
        filters=hp.Choice('filters_block4', [256, 512]), 
        kernel_size=hp.Choice('kernel_size_block4', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(
        filters=hp.Choice('filters_block4_2', [256, 512]), 
        kernel_size=hp.Choice('kernel_size_block4_2', [3, 5]), 
        padding='same'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Capa totalmente conectada
    x = layers.Dense(
        units=hp.Int('dense_units', 256, 512, step=64)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(hp.Float('dense_dropout', 0.4, 0.6, step=0.1))(x)
    
    # Predicción del Bounding Box
    box_output = layers.Dense(4, activation='sigmoid', name='box_output')(x)
    
    # Predicción de la clase del objeto
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    # Modelo final
    model = models.Model(inputs=input_image, outputs=[box_output, class_output])
    
    # Compilación del modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss={
            'box_output': 'mean_squared_error',
            'class_output': 'sparse_categorical_crossentropy'
        },
        metrics={
            'box_output': 'mean_absolute_error',
            'class_output': 'accuracy'
        }
    )
    
    return model


#buscar los mejores hiperparámetros
tuner = kt.Hyperband(
    build_model,
    max_epochs=20,
    directory='hyperparameter_search',
    project_name='cnn_color_model'
)

#dividir datos para entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_boxes, y_val_boxes, y_train_labels, y_val_labels = train_test_split(
    images, boxes, labels, test_size=0.2, random_state=42
)

# Callback para detener el entrenamiento temprano
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Ejecutar la búsqueda de hiperparámetros
tuner.search(
    X_train, {'box_output': y_train_boxes, 'class_output': y_train_labels},
    validation_data=(X_val, {'box_output': y_val_boxes, 'class_output': y_val_labels}),
    epochs=20,
    callbacks=[early_stopping]
)

# Obtener el mejor modelo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Mejores hiperparámetros encontrados: {best_hps.values}")

# Entrenar el mejor modelo
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    X_train, {'box_output': y_train_boxes, 'class_output': y_train_labels},
    validation_data=(X_val, {'box_output': y_val_boxes, 'class_output': y_val_labels}),
    epochs=20,
    callbacks=[early_stopping]
)

2024-11-21 21:01:40.279894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 21:01:40.296076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 21:01:40.313012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 21:01:40.318107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 21:01:40.331216: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1
Reloading Tuner from hyperparameter_search/cnn_color_model/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_conv_blocks
256               |64                |filters_0
3                 |3                 |kernel_size_0
0.4               |0.3               |dropout_0
256               |32                |filters_1
3                 |3                 |kernel_size_1
0.3               |0.2               |dropout_1
384               |256               |dense_units
0.4               |0.3               |dense_dropout
0.01              |0.01              |learning_rate
32                |32                |filters_2
3                 |3                 |kernel_size_2
0.4               |0.2               |dropout_2
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0            

2024-11-21 21:01:55.048637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13268 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:11:00.0, compute capability: 8.6


Epoch 1/3


I0000 00:00:1732222923.072083    1663 service.cc:146] XLA service 0x7fa5900151c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732222923.072115    1663 service.cc:154]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
2024-11-21 21:02:03.293478: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-21 21:02:03.617393: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-11-21 21:02:04.936416: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 35.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-21 21:02:04.953915: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [1]:
!pip freeze > requirements.txt

In [2]:
!python --version

Python 3.11.0rc1
